![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/ShowNow_logo.png)

# ShowNow Analytics Dashboard

You’ve just joined **ShopNow**, a fast-growing online retailer specializing in consumer electronics and lifestyle products.
The **business intelligence team** needs **real-time visibility** into platform activity to support marketing campaigns and flash sales.

They’ve asked you, as the **Analytics Engineer**, to design a data pipeline that provides:

1. **Real-time counters** for total orders, revenue, and active users
2. **Rolling KPIs** — for example, sales and sessions in the last 5 minutes
3. **Live leaderboards** of top-selling products and categories
4. **Cached metrics** such as *Average Order Value (AOV)* for fast dashboard loading
5. **An interactive Streamlit dashboard** to visualize and monitor all of the above

Your mission: build this entire end-to-end analytics system — from data collection to live visualization.

## Part I - Generate real-time data

For this project, we are going to mimic real users generating sales with Python.

<Note type="note">

In real life Redis will be connected to a web-app that will generate actual data.

</Note>

In a seperate `.py` create a script that will:

- randomly generate sales of the products below:
    - `Wireless Earbuds`
    - `Smartwatch`
    - `Gaming Keyboard`
    - `4K Monitor`
    - `Portable Speaker`
    - `Fitness Tracker`
    - `Mechanical Mouse`
    - `Bluetooth Headphones`
    - `Power Bank`
    - `Smart Home Hub`
- Update your redis database
- Sales should be generated infinitely over a random period of time each of 1 to 2 seconds


<Note type="hint">

- You should use the `random` library.
- Your project should look something like this:

```
├── project_directory
   ├── sales_data_generator.py
   └── analytics_pipeline.ipynb (or .py)
```

</Note>



In [1]:
#!pip install -q xgboost
#!pip install -q s3fs
#!pip install -U kaleido
#!pip install Boto3
#!pip install s3fs
# Load in our libraries
#pip install pymongo[srv]
#pip install dotenv
#%pip install redis streamlit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
# import s3fs
#import boto3
import time
import random
import redis

# plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.colors as pc

# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "colab" # pour que colab ne bloque pas l'export svg

import warnings
warnings.filterwarnings('ignore')



# montage drive
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!git clone https://github.com/Yakudawoo/shopnow_analytics.git

Cloning into 'shopnow_analytics'...


In [14]:
!git status


On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [11]:
!ls

In [15]:
!cp /content/00-Analytics_pipeline.ipynb .


cp: cannot stat '/content/00-Analytics_pipeline.ipynb': No such file or directory


# MAC

/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

brew update
brew install redis



#WINDOWS

In [4]:


# ======================
# Connexion Redis (COLAB SAFE avec userdata.get()
#à remplacer par os.getenv("") ou os.environ("")
# dans vscode)
# ======================

host = userdata.get("REDHOST")
port = userdata.get("REDPORT")
password = userdata.get("REDPASS")

print("REDHOST =", host)
print("REDPORT =", port)
print("REDPASS exists =", password is not None)

if not all([host, port, password]):
    raise RuntimeError("❌ Secrets Colab manquants (REDHOST / REDPORT / REDPASS)")

r = redis.Redis(
    host=host,
    port=int(port),
    password=password,
    decode_responses=True,
    socket_timeout=5
)

print("Redis ping:", r.ping())
print("Events:", r.xlen("sales:stream"))

# ======================
# Génération des ventes
# ======================

STREAM_KEY = "sales:stream"

PRODUCTS = [
    "Wireless Earbuds",
    "Smartwatch",
    "Gaming Keyboard",
    "4K Monitor",
    "Portable Speaker",
    "Fitness Tracker",
    "Mechanical Mouse",
    "Bluetooth Headphones",
    "Power Bank",
    "Smart Home Hub"
]

PRICES = {
    "Wireless Earbuds": 79.99,
    "Smartwatch": 199.99,
    "Gaming Keyboard": 129.99,
    "4K Monitor": 399.99,
    "Portable Speaker": 89.99,
    "Fitness Tracker": 149.99,
    "Mechanical Mouse": 59.99,
    "Bluetooth Headphones": 119.99,
    "Power Bank": 39.99,
    "Smart Home Hub": 179.99
}

print("📡 Génération de ventes en cours...")

while True:
    product = random.choice(PRODUCTS)
    quantity = random.randint(1, 3)
    total_amount = round(PRICES[product] * quantity, 2)

    event = {
        "product": product,
        "quantity": quantity,
        "amount": total_amount
    }

    r.xadd(STREAM_KEY, event)
    r.incr("total_orders")
    r.incrbyfloat("total_revenue", total_amount)
    r.zincrby("product_leaderboard", quantity, product)

    print(f"🛒 Vente : {product} x{quantity} → {total_amount}€")
    time.sleep(random.uniform(1, 2))


🛒 Vente : Power Bank x3 → 119.97€
🛒 Vente : Gaming Keyboard x3 → 389.97€
🛒 Vente : Smartwatch x3 → 599.97€
🛒 Vente : 4K Monitor x2 → 799.98€
🛒 Vente : Bluetooth Headphones x1 → 119.99€
🛒 Vente : Smartwatch x2 → 399.98€
🛒 Vente : 4K Monitor x2 → 799.98€
🛒 Vente : Gaming Keyboard x3 → 389.97€
🛒 Vente : Fitness Tracker x3 → 449.97€
🛒 Vente : 4K Monitor x2 → 799.98€
🛒 Vente : Smartwatch x2 → 399.98€
🛒 Vente : Wireless Earbuds x2 → 159.98€
🛒 Vente : Portable Speaker x1 → 89.99€
🛒 Vente : 4K Monitor x3 → 1199.97€
🛒 Vente : Power Bank x2 → 79.98€
🛒 Vente : Mechanical Mouse x1 → 59.99€
🛒 Vente : Bluetooth Headphones x1 → 119.99€
🛒 Vente : Bluetooth Headphones x3 → 359.97€
🛒 Vente : Fitness Tracker x2 → 299.98€
🛒 Vente : Power Bank x2 → 79.98€
🛒 Vente : Gaming Keyboard x2 → 259.98€
🛒 Vente : Bluetooth Headphones x1 → 119.99€
🛒 Vente : Bluetooth Headphones x3 → 359.97€
🛒 Vente : Fitness Tracker x3 → 449.97€
🛒 Vente : Gaming Keyboard x1 → 129.99€
🛒 Vente : 4K Monitor x1 → 399.99€
🛒 Vente : Gaming

KeyboardInterrupt: 

## Part II - Generate analytics

Keep your `.py` file running for the rest of the rest exercise.

- Look at [XRANGE](https://redis.io/docs/latest/commands/xrange/) documentation and iterate over the minimum and maximum possible ID and return a maximum of 2 entries.

* Now that you have a better understanding of `XRANGE`, build a function that will:
    * count the number of transactions
    * sum the total amount of revenue

<Note type="hint">

Look at:

* [INCR](https://redis.io/docs/latest/commands/INCR/)
* [INCRBYFLOAT](https://redis.io/docs/latest/commands/incrbyfloat/)

</Note>

* Output the current total transactions and total revenue

* Following the same method, create a leader of the best selling products

<Note type="hint">

This time:

- [`ZINCRBY`](https://redis.io/docs/latest/commands/zincrby/)
- [`ZRANGE`](https://redis.io/docs/latest/commands/zrange/)

will help you!

</Note>

* Now let’s track **recent activity**, not just totals. Find a way to get all the recent sales from the last 10 seconds

<Note type="hint">

- Again use `XRANGE`
- Another helper is `redis.Redis.time()`

</Note>

* We want to compute the Average Order Value (AOV) across all transactions. However at scale, this might involve joining large tables — expensive for real-time dashboards. Let’s cache the result in Redis.

<Note type="hint">

Use `SETEX` with short TTLs (1–5 min) for computed metrics that update often but not continuously.

</Note>

## Part III - Build a live dashboard

For that final part, you will need to build a live dashboard. Use any visualization library that you want. The goal is to have a line chart that gets updated every 10 seconds showing:

- Updated revenue
- Cumulative revenue

<Note type="note" title="in real life">

In real life you wouldn't build a dashboard on a notebook but using technologies like:

* Streamlit
* Dash
* PowerBI

This is for you to practice and see how powerful Redis already is.
</Note>

## Final Part

You reached the end of the exercise, make sure to stop your `sales_data_generator.py` before moving on to the next lectures!